### Config stuff

In [1]:
import random
import pyspark
from pyspark.sql import SparkSession, functions
import ConnectionConfig as cc
from pyspark.sql.functions import *
cc.setupEnvironment()
cc.listEnvironment()

ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\dobis\AppData\Roaming
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: VIKI
COMSPEC: C:\WINDOWS\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
GOPATH: C:\Users\dobis\go
HOMEDRIVE: C:
HOMEPATH: \Users\dobis
IGCCSVC_DB: AQAAANCMnd8BFdERjHoAwE/Cl+sBAAAAAdGjjLHLGEWRGn9vRsCSowQAAAACAAAAAAAQZgAAAAEAACAAAAD7UhTq8CVvkaUfJ5fXTR5kgkvcIed3OfwPabt1yHYIgAAAAAAOgAAAAAIAACAAAABSFZMhyRZv+fj9Q44MNd0sMMQVbnBwNGmcsxiNFFrAcmAAAAAQjo+0swEYFhn4kypkFiEe0Z+EUeRh+XkMWaxY6J5h885R6WUpGPQjsBjQtDBiDzTJJu/Eu8HKO9rNDQ2HtHCLXjOrbciSueB80zvNehaNnexWcFALkN4Q37FMwos4go9AAAAAsBkth/vA4x8SgTkTjgM6mv3GKidgi5oDWFMyb92y29Ab+MuztrSDRGCJMXxOGeO0p8LJ8WZBYOU66GQTsD0WFw==
IPY_INTERRUPT_EVENT: 2416
JPY_INTERRUPT_EVENT: 2416
JPY_PARENT_PID: 2412
JPY_SESSION_NAME: 05_2_DWH_dimSalesRep_Initial.ipynb
LANG: en_US.UTF-8
LANGUAGE: 
LC_ALL: en_US

In [2]:
spark = cc.startLocalCluster("dimSalesRepInit")
spark.getActiveSession()


# Creating the operational database
In order to run this demo you have to create a tutorial_op database and run the PostgreSQL_SalesOperational.sql script.


# Initial load
We will create a slowly changing dimension type 2 called dimSalesRep based on a sourceTable in our operational database called dbo.salesrep. SCD2  tables demand extra care and  because we will store hirstorical values of the dimension with the help of ranges.
This notebook will create the table and fill it with the initial data. A second notebook will be used for increments of new and changed data.

This is an example of the expected output (salesRepSK is different
```
+----------+-------------+-------------+-----------+-------------------+-------------------+--------------------+-------+
|salesRepID|         name|       office| salesRepSK|          scd_start|            scd_end|                 md5|current|
+----------+-------------+-------------+-----------+-------------------+-------------------+--------------------+-------+
|a46add1...|      Z. Jane|     New York|          0|1990-01-01 00:00:00|2100-12-12 00:00:00|303db545462092a92...|   true|
|s1fedf1...|   P. Chapman|       Berlin|          1|1990-01-01 00:00:00|2100-12-12 00:00:00|14b094c31bf9e4149...|   true|
|d5e6f77...|     T. Crane|     New York|          2|1990-01-01 00:00:00|2100-12-12 00:00:00|6c062f95defda9dc3...|   true|
```




In [3]:
cc.set_connectionProfile("tutorial_op")

df_operational_sales_rep = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "salesrep") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "salesRepID") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 20) \
    .load()

#Method 1 : Use the DataFrame API
#df_dim_sales_rep = df_sales_rep.withColumn("salesRepSK", expr("uuid()")) \
#    .withColumn("scd_start", lit("1990-01-01").cast("timestamp")) \
#    .withColumn("scd_end", lit("2100-12-12").cast("timestamp")) \
#    .withColumn("md5", md5(concat( col('name'), col('office')))) \
#    .withColumn("current", lit(True))

#Method 2 : Use SQL
df_operational_sales_rep.createOrReplaceTempView("dimSalesRep")
df_dim_sales_rep = spark.sql("select uuid() as salesRepSK, *, to_timestamp('1999-01-01','yyyy-MM-dd') as scd_start, to_timestamp('2100-12-12','yyyy-MM-dd') as scd_end, md5(concat( name, office)) as md5, True as current  from dimSalesRep")

df_dim_sales_rep.printSchema()
df_dim_sales_rep.show()
df_dim_sales_rep.write.format("delta").mode("overwrite").saveAsTable("dimSalesRep")

#spark.sql("ALTER TABLE dimSalesRep  ADD columns (salesRepSK long GENERATED ALWAYS AS IDENTITY (START WITH 0 INCREMENT BY 1)")

NoSectionError: No section: 'tutorial_op'

* The function lit() is used when you want a fixed column value for every row. In this case scd_start, scd_end and current.
* The function md5() performs a md5-hash function on the preferred columns. This is needed to detect scd2 changes. When one of the included columns changes, the md5-hash will change. Include all SCD2 columns in the md5-hash function.

## Delete the spark session

In [5]:
spark.stop()